In [1]:
print("chroma db")

chroma db


In [2]:
# !pip install chromadb-client==1.0.0
# !pip install --upgrade chromadb==1.0.0
#!pip install --upgrade langchain-chroma

In [3]:
# !pip install chromadb tiktoken transformers setence_transformers pypdf

In [4]:
# !pip install faiss-cpu

In [5]:
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_community.embeddings import HuggingFaceEmbeddings # type: ignore
from langchain_community.vectorstores import Chroma # type: ignore
from langchain_community.vectorstores import FAISS # type: ignore
from langchain_community.vectorstores.faiss import DistanceStrategy
from langchain_core.documents import Document # type: ignore
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import tiktoken
import warnings
warnings.filterwarnings("ignore")

C:\Users\euiyoung.hwang\Git_Workspace\Vector_DB_with_LLM\.venv\Lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.18) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [6]:
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['REQUESTS_CA_BUNDLE'] = ''

In [7]:
docs = [
            Document(page_content="사과", metadata=dict(page=1)),
            Document(page_content="애플", metadata=dict(page=1)),
            Document(page_content="바나나", metadata=dict(page=2)),
            Document(page_content="오렌지", metadata=dict(page=2)),
            Document(page_content="고양이", metadata=dict(page=3)),
            Document(page_content="야옹", metadata=dict(page=3)),
            Document(page_content="강아지", metadata=dict(page=4)),
            Document(page_content="멍멍", metadata=dict(page=4)),
            Document(page_content="해", metadata=dict(page=5)),
            Document(page_content="달", metadata=dict(page=5)),
            Document(page_content="물", metadata=dict(page=6)),
            Document(page_content="불", metadata=dict(page=6)),
            Document(page_content="apple", metadata=dict(page=7)),
]

In [8]:
embeddings = HuggingFaceEmbeddings(
        model_name="BAAI/bge-m3",
        model_kwargs = {'device': 'cpu'}, # 모델이 CPU에서 실행되도록 설정. GPU를 사용할 수 있는 환경이라면 'cuda'로 설정할 수도 있음
        encode_kwargs = {'normalize_embeddings': True}, # 임베딩 정규화. 모든 벡터가 같은 범위의 값을 갖도록 함. 유사도 계산 시 일관성을 높여줌
    )
# embeddings = HuggingFaceHubEmbeddings(huggingfacehub_api_token=HUGGINGFACE_API_KEY, 
#                                      repo_id="sentence-transformers/all-MiniLM-L12-v2")

In [9]:
tokenizer = tiktoken.get_encoding('cl100k_base')
def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [10]:
print("In Progressing..")

In Progressing..


In [11]:
# print(docs)

In [12]:
def get_initial_data():
    print("try to Loading get_initial_data")
    # -- Reference: https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
    ## pdf 파일로드 하고 쪼개기
    loader = PyPDFLoader('./gsat_170823.pdf')
    pages = loader.load_and_split()

    ## chunk로 쪼개기
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0,length_function=tiktoken_len)
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    docs = text_splitter.split_documents(pages)
    
    return docs

In [13]:
def Faiss_save_load(embeddings):
    dir_path = "./"
    store_name = dir_path + "Faiss_vector"

    print("try to Loading " + dir_path + "...")
    
    if os.path.exists(store_name):
        print(f"Load Faiss Vector..")
        return FAISS.load_local(store_name, embeddings, allow_dangerous_deserialization=True)
    
    # db = Chroma.from_documents(docs, embeddings)
    # Default: L2, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT -> Cosine Similariy
    db = FAISS.from_documents(get_initial_data(), embeddings, distance_strategy=DistanceStrategy.MAX_INNER_PRODUCT, normalize_L2 = True)
    
    db.save_local(store_name)
    print(f"Success to save FAISS index from ./{store_name}")
    
    return db

In [14]:
# text1 = 'simple text for FAISS testing'
# text2 = 'another simple text for vectorDB testing'

# db = FAISS.from_texts(
#     [text1, text2], embedding=embeddings,
#     ids = [str(uuid.uuid4()) for i in range(len(documents))],
#     metadatas=[{"source": "텍스트문서"}, {"source": "텍스트문서"}]
# )

In [15]:
db = Faiss_save_load(embeddings)

try to Loading ./...
Load Faiss Vector..


In [16]:
# -- 문서 저장소 ID 확인
# https://wikidocs.net/234014
db.index_to_docstore_id

{0: '8f70b74f-5efa-40a0-8a66-e0e08388d894',
 1: 'a7ef2161-8a37-48c7-a7fe-e8b81ce8c8aa',
 2: '9ef929fe-acef-4e84-9c64-e77fd8ab25e2',
 3: '7a37f12c-b4ad-4753-8fd9-9565652365da',
 4: '9b548a78-83c3-4e2c-b866-066069a0b2af',
 5: 'd90aa706-90e1-4413-8b53-e10057474226',
 6: '3c25fecb-c289-4f83-b84e-b2c2c6f9869d',
 7: 'c62abf7b-62bd-4020-b61f-54bddfdd55d5',
 8: '4dbc0050-4384-4afa-873d-f23220937af4',
 9: '3bb67c8b-7e10-4c6e-896d-e8087874df6f',
 10: '2f43b41b-bafe-43c8-8453-b6f35e7d66ab',
 11: '596d4092-c5a2-4680-a9a3-47df2a10ce45',
 12: 'acddd9a7-542b-4348-8ab8-e85e6eeed4f1',
 13: '7e388da8-bde9-4f8f-a5bc-64a94a8889f9',
 14: '0c3b11c7-d392-4726-a090-40f806ff82e7',
 15: '12bd0afb-479c-4303-b06b-558bdc281173',
 16: 'cf381fd0-3d5f-41e8-a51b-cf516696d952',
 17: 'ec47299d-79c2-4cf6-a103-5e27c130d7b4',
 18: 'e5544fcf-8596-45fb-abb6-13b9c2734b77',
 19: '62775e9d-a0c0-4a14-96dc-fbb90746daf0',
 20: '973d8b2b-9a53-4063-92a5-58c6f6e84b87',
 21: 'aaee1360-38f6-4338-9222-a721fda26b7a',
 22: '14ce6d93-4834-

In [17]:
# db.docstore._dict

In [18]:
question = '삼성전자의 주요 사업영역은?'
r_docs = db.similarity_search(question)
r_docs

[Document(metadata={'source': './gsat_170823.pdf', 'page': 9}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.\n⑴ Consumer Electronics (CE) 부문 \n① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 \n도전\n② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비\n③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발\n⑵ Information technology & Mobile communication (IM) 부문\n① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공\n② 네트워크 : 뛰어난 기술력과 성공적인 경험을 바탕으로 차세대 네트워크 인프라와 솔루션을 \n공급\n⑶ Device Solutions (DS) 부문\n① 메모리 : 세계 최고 성능의 최첨단 그린메모리로 더욱 편리한 세상을 향해 나아감'),
 Document(metadata={'source': './gsat_170823.pdf', 'page': 0}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분

In [19]:
print(r_docs[0].page_content)

11
Ⅱ 기업 상세 분석
1  사업분야(내용)
Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?
A
 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-
tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다.
⑴ Consumer Electronics (CE) 부문 
① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에 
도전
② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비
③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발
⑵ Information technology & Mobile communication (IM) 부문
① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
② 네트워크 : 뛰어난 기술력과 성공적인 경험을 바탕으로 차세대 네트워크 인프라와 솔루션을 
공급
⑶ Device Solutions (DS) 부문
① 메모리 : 세계 최고 성능의 최첨단 그린메모리로 더욱 편리한 세상을 향해 나아감


In [20]:
# 검색기로 변환
retriever = db.as_retriever()
# 검색 수행
retriever.invoke("삼성전자에 대하여 알려줘")
# retriever.invoke("삼성전자의 주요 사업영역은?")

[Document(metadata={'source': './gsat_170823.pdf', 'page': 0}, page_content='2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요'),
 Document(metadata={'source': './gsat_170823.pdf', 'page': 9}, page_content='11\nⅡ 기업 상세 분석\n1  사업분야(내용)\nQ1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ?\nA\n 삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica-\ntion) 사업부문, DS(Device Solutions) 사업

In [21]:
print("Done..")

Done..
